<a href="https://colab.research.google.com/github/muqadasuet41/data_optimzation/blob/main/data_optimize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cell 1: install requirements
!pip install pandas openpyxl streamlit-aggrid zipfile36

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 112.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 93.0 MB/s eta 0:00:00


In [3]:
# Cell 2: helper functions (a simplified subset of merge_tools)
import pandas as pd
from datetime import datetime

def parse_employee_data(x, lc):
    df = x.copy()
    # Possible layouts:
    # Layout A: columns ['Employee ID','Employee Name','Skill','Level']
    # Layout B: Name/ID in header and skills as columns
    if ('skill' in lc) and ('level' in lc):
        # straightforward row-wise format
        df = df.rename(columns={lc['skill']:'Skill', lc['level']:'Level'})
        # ensure name/id
        if 'employee name' in lc:
            df['Employee Name'] = df[lc['employee name']]
        if 'employee id' in lc:
            df['Employee ID'] = df[lc['employee id']]
        # keep relevant columns
        keep = [c for c in ['Employee ID','Employee Name','Skill','Level'] if c in df.columns]
        return df[keep]
    else:
        # try pivoting: first row contains name/id and rest columns are skill names with levels in row 2
        # We'll attempt to melt columns other than Name/ID
        possible_id = None
        for k in ['employee id','id','emp id','empid']:
            if k in lc:
                possible_id = lc[k]
        possible_name = None
        for k in ['employee name','name']:
            if k in lc:
                possible_name = lc[k]
        # If skills are columns, melt them
        non_skill_cols = [c for c in df.columns if c in [possible_id, possible_name]]
        skill_cols = [c for c in df.columns if c not in non_skill_cols]
        if len(skill_cols) == 0:
            raise ValueError('Unrecognized Excel format; no skill columns found')
        melted = df.melt(id_vars=non_skill_cols, value_vars=skill_cols, var_name='Skill', value_name='Level')
        if possible_name:
            melted = melted.rename(columns={possible_name:'Employee Name'})
        if possible_id:
            melted = melted.rename(columns={possible_id:'Employee ID'})
        return melted[[c for c in ['Employee ID','Employee Name','Skill','Level'] if c in melted.columns]]




def merge_into_master(master_df, employee_df, cycle_label=None):
    # master_df has columns: ['Employee ID','Employee Name','Skill','Level','Last Updated','Cycle']
    now = datetime.utcnow().isoformat()
    employee_df = employee_df.copy()
    if 'Level' not in employee_df.columns:
        raise ValueError('Employee data missing Level column')
    # fill missing metadata
    if 'Employee ID' not in employee_df.columns and 'Employee Name' in employee_df.columns:
        employee_df['Employee ID'] = employee_df['Employee Name']
    employee_df['Last Updated'] = now
    employee_df['Cycle'] = cycle_label
    # Upsert rows: if (Employee ID + Skill) exists, update Level and Last Updated
    master = master_df.copy()
    if master is None or master.empty:
        return employee_df[['Employee ID','Employee Name','Skill','Level','Last Updated','Cycle']]
    # build key
    master['_key'] = master['Employee ID'].astype(str) + '||' + master['Skill'].astype(str)
    employee_df['_key'] = employee_df['Employee ID'].astype(str) + '||' + employee_df['Skill'].astype(str)
    # remove duplicates in master by keeping latest
    master = master.drop_duplicates(subset=['_key'], keep='last')
    # for keys in employee_df, replace in master
    emp_keys = set(employee_df['_key'])
    master = master[~master['_key'].isin(emp_keys)]
    combined = pd.concat([master, employee_df], ignore_index=True)
    combined = combined.drop(columns=['_key'])
    return combined

In [5]:
# Cell 3: quick demo using sample excel files created on the fly
from io import BytesIO
import pandas as pd # Import pandas here as well


# create sample employee file (layout A)
emp_a = pd.DataFrame({'Employee Name':['Alice']*3, 'Employee ID':['E001']*3,'Skill':['PLC','HMI','Wiring'],'Level':[2,3,1]})
buf = BytesIO()
with pd.ExcelWriter(buf, engine='openpyxl') as w:
    emp_a.to_excel(w, index=False)
buf.seek(0)

# We need a dummy parse_employee_excel_bytes function or a way to load the excel data
# For this demo, let's assume parse_employee_data can handle bytes directly or we adapt it
# Assuming parse_employee_data can take a file-like object:
# parsed = parse_employee_data(buf, {'employee name': 'Employee Name', 'employee id': 'Employee ID', 'skill': 'Skill', 'level': 'Level'})

# Alternatively, read the excel back into a DataFrame and then parse
excel_data = pd.read_excel(buf)
parsed = parse_employee_data(excel_data, {'employee name': 'Employee Name', 'employee id': 'Employee ID', 'skill': 'Skill', 'level': 'Level'})


master = pd.DataFrame(columns=['Employee ID','Employee Name','Skill','Level','Last Updated','Cycle'])
master = merge_into_master(master, parsed, cycle_label='2025-Cycle1')
master

/tmp/ipython-input-379266170.py:49: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow().isoformat()


,Employee ID,Employee Name,Skill,Level,Last Updated,Cycle
0,E001,Alice,PLC,2,2025-10-20T07:38:27.540838,2025-Cycle1
1,E001,Alice,HMI,3,2025-10-20T07:38:27.540838,2025-Cycle1
2,E001,Alice,Wiring,1,2025-10-20T07:38:27.540838,2025-Cycle1


# New Section